In [ ]:
import torch

DEVICE = torch.device( 'cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = 'vit_h'

In [ ]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

In [ ]:
import os

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

In [ ]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

In [ ]:
mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.93,
    stability_score_thresh=0.98,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100,  # Requires open-cv to run post-processing
)

In [ ]:
import os

IMAGE_NAME = "images.jpeg"
IMAGE_PATH = os.path.join(HOME, "segmentation", IMAGE_NAME)

In [ ]:
import cv2
import supervision as sv

image_bgr = cv2.imread(IMAGE_PATH)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

sam_result = mask_generator.generate(image_rgb)

In [ ]:
print(sam_result[0].keys())

In [ ]:
mask_annotator = sv.MaskAnnotator(color_lookup=sv.ColorLookup.INDEX)

detections = sv.Detections.from_sam(sam_result=sam_result)

annotated_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)

sv.plot_images_grid(
    images=[image_bgr, annotated_image],
    grid_size=(1, 2),
    titles=['source image', 'segmented image']
)

In [ ]:
print(len(sam_result))

In [ ]:
masks = [
    mask['segmentation']
    for mask
    in sorted(sam_result, key=lambda x: x['area'], reverse=True)
]

sv.plot_images_grid(
    images=masks,
    grid_size=(rows, cols),
    size=(16, 16)
)

In [ ]:
import math

# Number of images
num_images = len(masks)

# Calculate the number of rows and columns
rows = math.ceil(num_images / 10)  # Ensure enough rows to fit all images
cols = min(10, num_images)  # Maximum 10 columns, adjust if fewer images

In [ ]:
mask_predictor = SamPredictor(sam)

In [ ]:
import cv2

# 'sam_result' is a list of dicts, one per detected mask.
# Each dict has keys like 'segmentation', 'bbox', 'area', etc.
# For example: sam_result[i]['bbox'] might look like [x, y, w, h]

# Convert the image back to BGR if needed (it already is in BGR as `image_bgr` from your code).
output_image = image_bgr.copy()

# Iterate over all masks and draw their bounding boxes
for mask_data in sam_result:
    bbox = mask_data['bbox']  # [x, y, w, h]
    
    # Convert coordinates to int
    x, y, w, h = map(int, bbox)
    
    cv2.rectangle(
        img=output_image,
        pt1=(x, y),
        pt2=(x + w, y + h),
        color=(0, 255, 0),
        thickness=2
    )

# Optionally, you can display the image using OpenCV
# Note: if you're running in a notebook or a headless environment, 
# you might need to use matplotlib or other methods to display the image.
cv2.imshow("Objects with Bounding Boxes", output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [16]:
import os

IMAGE_NAME = "dog.jpeg"
IMAGE_PATH = os.path.join(HOME, "data", IMAGE_NAME)

In [ ]:
import numpy as np
import supervision as sv
import cv2

# Extract masks from the SAM result and sort them by area (descending)
masks = [
    mask_data['segmentation']
    for mask_data in sorted(sam_result, key=lambda x: x['area'], reverse=True)
]

# Convert the masks to bounding boxes in xyxy format
xyxy_boxes = sv.mask_to_xyxy(masks=masks)

# Create a Detections object from the bounding boxes and masks
detections = sv.Detections(
    xyxy=xyxy_boxes,
    mask=masks
)

# (Optional) If you only want to display the largest detected object,
# filter detections by their area to select the largest one.
detections = detections[detections.area == np.max(detections.area)]

# Create annotators for boxes and masks
box_annotator = sv.BoxAnnotator(color=sv.Color.RED, color_lookup=sv.ColorLookup.INDEX)
mask_annotator = sv.MaskAnnotator(color=sv.Color.RED, color_lookup=sv.ColorLookup.INDEX)

# Annotate the original image with bounding boxes and masks
source_image = box_annotator.annotate(scene=image_bgr.copy(), detections=detections)
segmented_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)

# Visualize the annotated results side-by-side in a grid
sv.plot_images_grid(
    images=[source_image, segmented_image],
    grid_size=(1, 2),
    titles=['Source Image', 'Segmented Image']
)


AttributeError: 'list' object has no attribute 'shape'

NameError: name 'image_bgr' is not defined